# Processo de ETL para Dados de Licitações e Contratos do TCU

Esta seção descreve o processo ETL para o arquivo "Informativo de Licitações e Contratos" do Tribunal de Contas da União (TCU). O processo envolve a extração dos dados a partir do site do TCU, armazenamento em um bucket S3, e o processamento na camada `raw` para ingestão posterior.

## Visão Geral do ETL

O ETL (Extract, Transform, Load) consiste nas seguintes etapas principais:

1. **Extract (Extração)**: Uma função Lambda é utilizada para baixar o arquivo diretamente do site de jurisprudência do TCU. O arquivo em formato CSV é baixado e enviado para o bucket S3 em uma camada `raw`.
2. **Transform (Transformação)**: Na camada `raw`, o arquivo CSV é pré-processado, incluindo verificações de qualidade e formatação dos dados.
3. **Load (Carregamento)**: Os dados transformados são carregados na camada `trusted` do Data Lake para posteriores análises e processamento de relatórios.

## Estrutura do Arquivo

O arquivo "Informativo de Licitações e Contratos" possui a seguinte estrutura:

| Nome do Campo    | Formato | Descrição                                                                                                      |
|------------------|---------|------------------------------------------------------------------------------------------------------------------|
| `KEY`            | Texto   | Identificador único para o registro.                                                                            |
| `COLEGIADO`      | Texto   | Colegiado responsável pelo julgamento do Acórdão.                                                               |
| `TEXTOACORDAO`   | Texto   | Indicação do Acórdão que originou o enunciado.                                                                  |
| `ENUNCIADO`      | Texto   | Retrata o entendimento contido na deliberação da qual foi extraído, não constituindo resumo oficial do Tribunal. |
| `NUMERO`         | Número  | Número da publicação.                                                                                           |
| `TEXTOINFO`      | Texto   | Descrição detalhada do enunciado.                                                                               |
| `TITULO`         | Texto   | Título do informativo contendo número e ano.                                                                    |

## Fluxo Resumido do Processo ETL

1. **Download** do arquivo do TCU pela Lambda.
2. **Envio** do arquivo para o bucket S3 (camada `raw`).
3. **Transformação** dos dados na camada `raw`.
4. **Carga** dos dados processados em um Data Lake.

## Tecnologias Utilizadas

- **AWS Lambda**: Para download e armazenamento no S3.
- **AWS S3**: Armazenamento do arquivo CSV.
- **PySpark/Databricks**: Para transformação e limpeza dos dados.

## Exemplo de Uso dos Dados

Os dados armazenados e transformados podem ser utilizados para:

- Monitoramento e análise de deliberações em licitações e contratos.
- Geração de relatórios detalhados sobre entendimentos jurídicos do TCU.
- Insights para conformidade e boas práticas em processos licitatórios.


In [0]:
from pyspark.sql.functions import udf, regexp_extract, split, regexp_replace
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.utils import AnalysisException
from py4j.protocol import Py4JJavaError
from datetime import datetime
import urllib
import os

In [0]:
dt_load = datetime.today().strftime('%Y%m%d')

In [0]:
ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_ACCESS_KEY_ID")
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")
AWS_S3_BUCKET = os.getenv("S3_BUCKET_NAME")


In [0]:
layer = "raw"
final_layer = "trusted"
area = "tcu"
context = "boletim-informativo-lc"
file_name = "boletim-informativo-lc"
file_format = "parquet"
write_mode = "overwrite"

In [0]:
MOUNT_NAME = f"/mnt/{AWS_S3_BUCKET}"
SOURCE_URL = f"s3n://{ACCESS_KEY}:{ENCODED_SECRET_KEY}@{AWS_S3_BUCKET}"

if not any(mount.mountPoint == f"/mnt/{AWS_S3_BUCKET}" for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
else:
    print("Diretório já montado.")


Diretório já montado.


In [0]:
file_type = "csv"
infer_schema = "true"
first_row_is_header = "true"
delimiter = "|"
file_location = f"/mnt/{AWS_S3_BUCKET}/{layer}/{context}/{dt_load}/{file_name}.{file_type}"
df_raw = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).option("sep", delimiter).load(file_location)

In [0]:
df_renamed = df_raw.withColumnRenamed('KEY','informativo_acordao').withColumnRenamed('TITULO','titulo_acordao').withColumnRenamed('COLEGIADO','colegiado_acordao').withColumnRenamed('TEXTOACORDAO','texto_acordao').withColumnRenamed('ENUNCIADO','enunciado_acordao').withColumnRenamed('NUMERO','numero_acordao').withColumnRenamed('TEXTOINFO','infos_acordao')

In [0]:
df_split = df_renamed.withColumn("tag_acordao", split(df_renamed["texto_acordao"], ", ")[0]).withColumn("tipo_acordao", split(df_renamed["texto_acordao"], ", ")[1]).withColumn("relator_acordao", split(df_renamed["texto_acordao"], ", ")[2])

In [0]:
df_apply_regex = df_split.withColumn("ano_acordao", regexp_extract("tag_acordao", 'ano=""([^"]+)""', 1))

In [0]:
df_treated = df_apply_regex.withColumn("relator_acordao", regexp_replace('relator_acordao','"', ''))

In [0]:
df = df_treated.fillna({
  'tipo_acordao': 'Não disponível',
  'relator_acordao': 'Desconhecido',
  'ano_acordao': '1900',
  })

display(df.head(5))

informativo_acordao titulo_acordao colegiado_acordao texto_acordao enunciado_acordao numero_acordao infos_acordao tag_acordao tipo_acordao relator_acordao ano_acordao INFORMATIVO-LC-13325-0 Informativo de Licitações e Contratos 478/2024 Plenário " Acórdão 465/2024 Plenário , Representação, Relator Ministro-Substituto Augusto Sherman" O critério definido no art. 59, § 4º, da Lei 14.133/2021 conduz a uma presunção relativa de inexequibilidade de preços, devendo a Administração, nos termos do art. 59, § 2º, da referida lei, dar à licitante a oportunidade de demonstrar a exequibilidade de sua proposta. 478 "Representação formulada ao TCU apontou possíveis irregularidades na Concorrência 1/2023, do tipo menor preço e com modo de disputa aberto, realizada pela Universidade Federal Rural de Pernambuco (UFRPE), sob a regência da Lei 14.133/2021 e tendo por objeto a ""contratação de serviços especiais de engenharia relacionados à realização de planejamento, levantamentos, ensaios e a elaboração dos projetos executivos de engenharia, arquitetura e documentações legais referentes à construção do Campus definitivo da Unidade Acadêmica de Belo Jardim (UABJ)"". A sessão pública de recebimento e abertura de propostas, bem como de disputa de lances, contara com a participação de 31 empresas, sendo que as dezoito primeiras colocadas tiveram suas propostas desclassificadas por suposta inexequibilidade, em razão de haverem ofertado valor inferior a 75% do orçamento-base da licitação. Em face disso, a representante alegou, em essência, que: i) ""sua desclassificação teria sido feita de maneira sumária, sem que tenham sido promovidas as diligências necessárias previstas no art. 59, § 2º, da Lei 14.133/2021 e no próprio edital de licitação (subitem 6.10) para fins de demonstração da exequibilidade das melhores propostas apresentadas pelos licitantes""; ii) o valor proposto pela empresa vencedora teria sido 77% superior ao da proposta mais vantajosa, ""a qual teria sido desclassificada de maneira sumária, sem que tenha sido feita qualquer diligência""; iii) ""o entendimento jurisprudencial dominante pelo Poder Judiciário e pelo TCU seria no sentido de que é relativa e não absoluta a presunção de inexequibilidade das propostas inferiores a 75% do valor orçado pela Administração, cabendo, conforme o art. 59, § 2º, da Lei 14.133/2021, facultar às licitantes a demonstração da exequibilidade de suas propostas"". A partir do exame dos documentos e das informações relativas à Concorrência 1/2023, a unidade técnica entendeu que, de fato, as desclassificações das propostas apresentadas pelas licitantes ocorreram de forma sumária, sem que fosse dada oportunidade para que as empresas se manifestassem sobre a sua exequibilidade, o que, em tese, não seria procedimento condizente com o art. 59, § 2º, da Lei 14.133/2021, segundo o qual ""a Administração poderá realizar diligências para aferir a exequibilidade das propostas ou exigir das licitantes sua demonstração"". A unidade instrutiva também afirmou que a ausência de oportunidade para as licitantes demonstrarem a exequibilidade de suas propostas seria procedimento contrário à jurisprudência do Tribunal, ""conforme a Súmula TCU 262 e Acórdãos 1244/2018-TCU-Plenário; 2528/2012-TCU-Plenário; 1079/2017-TCU-Plenário; e 1161/2014-TCU-Plenário"". Realizada a oitiva prévia da UFRPE, esta informou haver retornado o certame à fase de julgamento das propostas e realizado diligências, junto às empresas que apresentaram propostas com valores inferiores a 75% do orçamento estimativo da licitação, para demonstração de sua exequibilidade, em cumprimento ao disposto no art. 59, § 2º, da Lei 14.133/2021. Dessa forma, a unidade técnica concluiu que a representação perdera seu objeto, propondo então que ela fosse considerada prejudicada, com o consequente arquivamento dos autos. Em seu voto, não obstante anuir à conclusão de que a representação ""perdeu seu objeto, ante o saneamento da irregularidade"", o relator julgou pertinente

In [0]:
df.write.save(f"/mnt/{AWS_S3_BUCKET}/{final_layer}/{area}/{context}/{dt_load}/{file_name}", format=file_format,mode=write_mode)